In [276]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from datetime import timedelta, datetime
import time

In [312]:
class CoinNamePreprocessing:
    def __init__(self, wallet_address):
        self.wallet_address = wallet_address

    def fetch_spot_meta(self):
        """Tname : coin name을 1:1로 매칭시키는 함수"""
        api_url = "https://api.hyperliquid.xyz/info"
        headers = {"Content-Type": "application/json"}
        payload = {"type": 'spotMetaAndAssetCtxs'}
        
        response = requests.post(api_url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        
        data = response.json()
        
        tokens = pd.DataFrame(data[0]["tokens"]).rename(columns={"name": "Tname"})
        universe = pd.DataFrame(data[0]["universe"]).rename(columns={"name": "coin"})
        
        tokens = tokens.merge(universe, left_on="index", right_on="index", how="inner")
        return tokens
    
    def fetch_and_pnl(self, time_interval=7, coin_name="HYPE", spot_only=False):
        """
        특정 기간(time_interval)의 거래 내역을 불러오고
        closedPnl을 float로 변환 후 코인별 누적 PnL을 계산
        특정 코인(coin_name)에 대해 필터링하여 그래프로 시각화
        """
        url = "https://api.hyperliquid.xyz/info"
        headers = {"Content-Type": "application/json"}   
        
        payload = {
            "type": "userFillsByTime",
            "user": self.wallet_address,
            "startTime": int(time.time() * 1000) - (time_interval * 86400000),
            "endTime": int(time.time() * 1000),
            "aggregateByTime": False
        }
        
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        
        if response.status_code != 200:
            print("오류:", response.status_code, response.text)
            return pd.DataFrame()
        
        data = response.json()
        df = pd.DataFrame(data)
        print(df)
        if "time" in df.columns:
            df["Timestamp"] = pd.to_datetime(df["time"], unit="ms")
            
        else:
            print("⚠️ Warning: 'time' 컬럼이 없음")
            return pd.DataFrame()
        
        if spot_only:
            filtered_df = df[df["coin"].str.startswith("@")] .copy()
            spot_meta = self.fetch_spot_meta()
            spot_meta1 = spot_meta[["Tname", "coin"]]
            coin_to_tname = dict(zip(spot_meta1["coin"], spot_meta1["Tname"]))
            filtered_df["coin"] = filtered_df["coin"].map(coin_to_tname)
            filtered_df = filtered_df[filtered_df["coin"].isin(spot_meta1["Tname"].values)]
        else:
            filtered_df = df[~df["coin"].str.startswith("@")] .copy()
        
        filtered_df["closedPnl"] = pd.to_numeric(filtered_df["closedPnl"], errors="coerce").fillna(0.0)
        filtered_df["Cumulative PnL"] = filtered_df.groupby("coin")["closedPnl"].cumsum()


        return filtered_df

In [333]:
import os
import pandas as pd
from PIL import Image, ImageDraw, ImageFont

def generate_pnl_card(output_path, eth_address, df_trades, up_image_path, down_image_path, icon_path=None):
    """
    코인 거래 데이터를 기반으로 PnL 카드 생성

    Args:
        output_path (str): 생성된 이미지 저장 경로
        eth_address (str): 사용자의 이더리움 지갑 주소
        df_trades (pd.DataFrame): 거래 데이터 (coin, closedPnl, px, sz)
        up_image_path (str): PnL이 양수일 때 사용할 배경 이미지
        down_image_path (str): PnL이 음수일 때 사용할 배경 이미지
        icon_path (str, optional): 아이콘 이미지 경로 (ex: HypurrQuant 로고)
    """
    ############################ rebalacing code 가지고 오면 이 부분만 수정 하면 됨 ############################ -> 각 변수는 알맞게 수정 하면 될듯함.
    # 종목별 PnL 및 총 PnL 계산
    df_pnl = df_trades.groupby("coin")["closedPnl"].sum().reset_index()
    df_pnl.columns = ["Coin", "Total PnL ($)"]
    total_pnl = df_pnl["Total PnL ($)"].sum()

    # 총 투자금 계산
    df_trades["investment"] = df_trades["px"].astype("float") * df_trades["sz"].astype("float")  # 각 거래별 투자금
    total_investment = df_trades["investment"].sum()
    roi_percent = (total_pnl / total_investment) * 100 if total_investment > 0 else 0

    # 가장 높은 PnL 기록한 종목 찾기
    max_pnl_coin = df_pnl.loc[df_pnl["Total PnL ($)"].idxmax(), "Coin"] if not df_pnl.empty else "N/A"
    ############################ rebalacing code 가지고 오면 이 부분만 수정 하면 됨 ############################
    # PnL이 양수이면 up 이미지, 음수이면 down 이미지 사용
    image_path = up_image_path if total_pnl >= 0 else down_image_path
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"{image_path} 파일이 존재하지 않습니다!")

    # 이미지 불러오기
    img = Image.open(image_path)
    draw = ImageDraw.Draw(img)
    img_width, img_height = img.size

    # 폰트 설정
    font_path = "/Library/Fonts/noto/NotoSans-Bold.ttf"
    font_pnl_title = ImageFont.truetype(font_path, 50)  # "Your Portfolio PnL"
    font_pnl_value = ImageFont.truetype(font_path, 200)  # ROI(%)
    font_header = ImageFont.truetype(font_path, 40)  # "Top Performing" & "Total Investment"
    font_value = ImageFont.truetype(font_path, 36)  # 종목명 및 투자금
    font_address = ImageFont.truetype(font_path, 20)  # 지갑 주소
    font_bottom = ImageFont.truetype(font_path, 20)  # 하단 문구

    # 정렬을 위한 위치 설정
    padding = 50
    y_offset = 200  # 모든 요소 아래로 이동

    # ROI(%) 및 PnL 색상 지정
    pnl_value_color = (255, 128, 128) if roi_percent < 0 else (173, 216, 230)

    # "Your Portfolio PnL" 표시
    title_x, title_y = padding + 50, 50 + y_offset
    draw.text((title_x, title_y), "Your Portfolio PnL", fill=(255, 255, 255), font=font_pnl_title)

    # 아이콘 추가 (선택 사항)
    if icon_path and os.path.exists(icon_path):
        icon = Image.open(icon_path).convert("RGBA").resize((50, 50))
        img.paste(icon, (title_x - 70, title_y + 5), icon)

    # ROI(%) 값 표시
    y_pnl_value = 180 + y_offset
    draw.text((title_x, y_pnl_value), f"{roi_percent:.2f}%", fill=pnl_value_color, font=font_pnl_value)

    # "Top Performing" & "Total Investment" 배치
    y_position = y_pnl_value + 250
    text_max_pnl = "Top Performing"
    text_investment = "Total Investment"
    text_max_pnl_value = max_pnl_coin
    text_investment_value = f"${total_investment:,.2f}"

    # 간격 조정
    spacing = 350  # 기존 200 → 350으로 증가
    max_pnl_width = draw.textbbox((0, 0), text_max_pnl, font=font_header)[2]
    investment_width = draw.textbbox((0, 0), text_investment, font=font_header)[2]

    # 중앙 정렬
    total_width = max_pnl_width + spacing + investment_width
    start_x = (img_width - total_width) // 2

    # 제목 출력 (간격 확장)
    draw.text((start_x, y_position+100), text_max_pnl, fill=(255, 255, 255), font=font_header)
    draw.text((start_x + max_pnl_width + spacing, y_position+100), text_investment, fill=(255, 255, 255), font=font_header)

    # 종목명 & 투자금 (아래로 정렬)
    y_value_position = y_position + 50
    max_pnl_value_width = draw.textbbox((0, 0), text_max_pnl_value, font=font_value)[2]
    investment_value_width = draw.textbbox((0, 0), text_investment_value, font=font_value)[2]

    draw.text((start_x + (max_pnl_width - max_pnl_value_width) // 2, y_value_position+100),
              text_max_pnl_value, fill=pnl_value_color, font=font_value)

    draw.text((start_x + max_pnl_width + spacing + (investment_width - investment_value_width) // 2, y_value_position+100),
              text_investment_value, fill=pnl_value_color, font=font_value)

    # 지갑 주소 & 하단 문구 출력
    draw.text((img_width - 650, img_height - 50), f"ETH address: {eth_address}", fill=(255, 255, 255), font=font_address)
    draw.text((padding, img_height - 50), "Created by hypurrQuant.xyz", fill=(255, 255, 255), font=font_bottom)

    # 이미지 저장 및 출력
    img.show()
    img.save(output_path)
    print(f"이미지 출력 완료: {output_path}")

In [334]:
wallet_address = "0x157a44B0555B31A0642fd0aF47f6806D3b86Ec9f"  
coin_processor = CoinNamePreprocessing(wallet_address)

df_orders = coin_processor.fetch_and_pnl(
    time_interval=31, 
    coin_name="", # 이 친구는 @를 포함할때 별 필요 없음 ""으로 해도 무방함 -> 현재는 현물로 접근함.
    spot_only=True 
)

       coin          px          sz side           time startPosition  \
0       BTC    100367.0     0.03527    A  1737957994621      -0.53821   
1       BTC    101893.0     0.03346    A  1737989495469      -0.57348   
2       BTC    100919.0     0.03346    B  1737991211118      -0.60694   
3        @1  103.453255  0.00953412    A  1738022400068    0.00953412   
4      HYPE      22.623        22.1    B  1738503395901           0.0   
...     ...         ...         ...  ...            ...           ...   
1433   HYPE      25.787         5.0    A  1739795680710        107.97   
1434   HYPE      25.786       38.78    A  1739795680710        102.97   
1435   HYPE      25.782       26.92    A  1739795680710         64.19   
1436   HYPE      25.781       37.27    A  1739795680710         37.27   
1437  TRUMP      17.736       277.6    A  1739795681997         277.6   

                       dir    closedPnl  \
0               Open Short          0.0   
1               Open Short          0

In [335]:
generate_pnl_card(
    output_path="/Users/ijongseung/pnlc/output_image.png",
    eth_address="0x157a44B0555B31A0642fd0aF47f6806D3b86Ec9f",
    df_trades=df_orders,
    up_image_path="/Users/ijongseung/pnlc/up.jpeg",
    down_image_path="/Users/ijongseung/pnlc/down.jpeg",
    icon_path="/Users/ijongseung/pnlc/hyperliquid_img.png"
)

이미지 출력 완료: /Users/ijongseung/pnlc/output_image.png
